In [1]:
import pandas as pd
import pyreadstat as st
pd.set_option('display.max_columns', None)
path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2025\2025 2T\[MT08-MT13] Microeconometría II\Clases prácticas\PS\PS3-20251111\Panel101.dta"

df, meta = st.read_dta(path)
df.head(1)

,country,year,y,y_bin,x1,x2,x3,opinion,op,Y,time,treated,did,_diff
0,1,1990,1.342788e+09,1.0,0.277904,-1.107956,0.282554,1.0,1.0,1.342788,0.0,0.0,0.0,0.0


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  70 non-null     int64  
 1   year     70 non-null     int64  
 2   y        70 non-null     float64
 3   y_bin    70 non-null     float64
 4   x1       70 non-null     float64
 5   x2       70 non-null     float64
 6   x3       70 non-null     float64
 7   opinion  70 non-null     float64
 8   op       70 non-null     float64
 9   Y        70 non-null     float64
 10  time     70 non-null     float64
 11  treated  70 non-null     float64
 12  did      70 non-null     float64
 13  _diff    70 non-null     float64
dtypes: float64(12), int64(2)
memory usage: 7.8 KB


In [3]:
meta.column_names_to_labels

{'country': 'Country',
 'year': 'Year',
 'y': 'Outcome Y',
 'y_bin': 'Binary outcome Y',
 'x1': 'Predictor x1',
 'x2': 'Predictor x2',
 'x3': 'Predictor x3',
 'opinion': 'Categorical variable',
 'op': None,
 'Y': None,
 'time': None,
 'treated': None,
 'did': None,
 '_diff': 'Diff-in-diff'}

In [5]:
df.describe()

,country,year,y,y_bin,x1,x2,x3,opinion,op,Y,time,treated,did,_diff
count,70.000000,70.00000,7.000000e+01,70.000000,70.000000,70.000000,70.000000,70.000000,70.00000,70.000000,70.000000,70.000000,70.000000,70.000000
mean,4.000000,1994.50000,1.845072e+09,0.800000,0.648001,0.133869,0.761851,2.442857,0.50000,1.845072,0.600000,0.428571,0.257143,0.257143
std,2.014441,2.89302,3.015167e+09,0.402888,0.468070,1.371099,1.446332,1.137448,0.50361,3.015167,0.493435,0.498445,0.440215,0.440215
min,1.000000,1990.00000,-7.863483e+09,0.000000,-0.567575,-1.621761,-1.165387,1.000000,0.00000,-7.863483,0.000000,0.000000,0.000000,0.000000
25%,2.000000,1992.00000,2.465716e+08,1.000000,0.328960,-1.215649,-0.079315,1.000000,0.00000,0.246572,0.000000,0.000000,0.000000,0.000000
50%,4.000000,1994.50000,1.897898e+09,1.000000,0.641262,-0.462143,0.514191,2.500000,0.50000,1.897898,1.000000,0.000000,0.000000,0.000000
75%,6.000000,1997.00000,3.372262e+09,1.000000,1.095843,1.607754,1.154864,3.000000,1.00000,3.372262,1.000000,1.000000,0.750000,0.750000
max,7.000000,1999.00000,8.941232e+09,1.000000,1.446412,2.530277,7.168922,4.000000,1.00000,8.941232,1.000000,1.000000,1.000000,1.000000


In [6]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Parto de tu df original
df_reg = df.copy()

# ==== Generar las variables como en Stata ====

# Y = y / 1e9
df_reg["Y"] = df_reg["y"] / 1_000_000_000

# time = (year >= 1994) & !missing(year)
df_reg["time"] = ((df_reg["year"] >= 1994) & df_reg["year"].notna()).astype(int)

# treated = (country > 4) & !missing(country)
df_reg["treated"] = ((df_reg["country"] > 4) & df_reg["country"].notna()).astype(int)

# did = time * treated
df_reg["did"] = df_reg["time"] * df_reg["treated"]

# ==== Misma muestra que Stata: sin missing en las variables del modelo ====
vars_modelo = ["Y", "time", "treated", "did"]
df_reg = df_reg.dropna(subset=vars_modelo)

# ==== Regresión OLS: Y time treated did ====
X = df_reg[["time", "treated", "did"]]
X = sm.add_constant(X)  # agrega _cons
y = df_reg["Y"]

model = sm.OLS(y, X)
res = model.fit()

print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     1.984
Date:                Fri, 14 Nov 2025   Prob (F-statistic):              0.125
Time:                        18:43:26   Log-Likelihood:                -173.06
No. Observations:                  70   AIC:                             354.1
Df Residuals:                      66   BIC:                             363.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3581      0.738      0.485      0.6

In [10]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from linearmodels.panel import PanelOLS

# ==========================
# 1) Partimos de tu df
# ==========================
df_fe = df.copy()

# Y = y / 1e9  (como en Stata: gen Y = y/1000000000)
df_fe["Y"] = df_fe["y"] / 1_000_000_000

# time = (year >= 1994) & !missing(year)
df_fe["time"] = ((df_fe["year"] >= 1994) & df_fe["year"].notna()).astype(int)

# treated = (country > 4) & !missing(country)
df_fe["treated"] = ((df_fe["country"] > 4) & df_fe["country"].notna()).astype(int)

# did = time * treated
df_fe["did"] = df_fe["time"] * df_fe["treated"]

# ==========================
# 2) Definimos el panel
# ==========================
# xtset country year  <->  índice multinivel (country, year)
df_fe = df_fe.set_index(["country", "year"])

# ==========================
# 3) Modelo FE (within) tipo: xtreg ..., fe
# ==========================
y = df_fe["Y"]

# Incluimos time, treated y did como en Stata + constante
# (treated será omitida por colinealidad con los efectos fijos)
X = df_fe[["time", "treated", "did"]]
X = sm.add_constant(X)

# entity_effects=True  -> efectos fijos por país (country)
mod_fe = PanelOLS(y, X, entity_effects=True, drop_absorbed=True)

# Para replicar Stata, usá var/cov clásica (no robusta):
res_fe = mod_fe.fit(cov_type="unadjusted")

print(res_fe.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                      Y   R-squared:                        0.0968
Estimator:                   PanelOLS   R-squared (Between):             -0.5101
No. Observations:                  70   R-squared (Within):               0.0968
Date:                Fri, Nov 14 2025   R-squared (Overall):             -0.0035
Time:                        18:59:03   Log-likelihood                   -166.19
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      3.2697
Entities:                           7   P-value                           0.0448
Avg Obs:                      10.0000   Distribution:                    F(2,61)
Min Obs:                      10.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             3.2697
                            

C:\Users\HP\AppData\Local\Temp\ipykernel_32968\1368542021.py:43: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

treated

  res_fe = mod_fe.fit(cov_type="unadjusted")


In [11]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats

def did_diff_table(df, yvar="Y", treated="treated", time="time"):
    df = df.copy()

    # ===============================
    # A. Medias por grupo y período
    # ===============================
    mean_before_control  = df[(df[treated]==0) & (df[time]==0)][yvar].mean()
    mean_before_treated  = df[(df[treated]==1) & (df[time]==0)][yvar].mean()
    diff_before = mean_before_treated - mean_before_control

    mean_after_control   = df[(df[treated]==0) & (df[time]==1)][yvar].mean()
    mean_after_treated   = df[(df[treated]==1) & (df[time]==1)][yvar].mean()
    diff_after = mean_after_treated - mean_after_control

    # ===============================
    # B. Regresión: Y = b0 + time + treated + did
    # ===============================
    df["did"] = df[treated] * df[time]

    X = df[[time, treated, "did"]]
    X = sm.add_constant(X)
    y = df[yvar]

    model = sm.OLS(y, X).fit()

    b_time, b_treated, b_did = model.params[time], model.params[treated], model.params["did"]
    se_time, se_treated, se_did = model.bse[time], model.bse[treated], model.bse["did"]

    # ===============================
    # C. Errores std de las "diff"
    # ===============================
    # before diff = treated - control  = β_treated
    se_before = model.bse["treated"]
    t_before = model.tvalues["treated"]
    p_before = model.pvalues["treated"]

    # after diff  = (β_treated + β_did)
    diff_after_coef = b_treated + b_did
    # Var(a+b) = Var(a)+Var(b)+2Cov(a,b)
    cov = model.cov_params().loc["treated","did"]
    se_after = np.sqrt(model.bse["treated"]**2 + model.bse["did"]**2 + 2*cov)
    t_after = diff_after_coef / se_after
    df_resid = int(model.df_resid)
    p_after = 2*(1-stats.t.cdf(abs(t_after), df_resid))

    # DID effect = β_did
    did_coef = b_did
    did_se = se_did
    did_t = model.tvalues["did"]
    did_p = model.pvalues["did"]

    # ===============================
    # D. Construcción de tabla
    # ===============================
    tabla = f"""
DIFFERENCE-IN-DIFFERENCES ESTIMATION RESULTS
Number of observations in the DIFF-IN-DIFF: {len(df)}

            Before       After
Control:    {len(df[(df[treated]==0) & (df[time]==0)])}     {len(df[(df[treated]==0) & (df[time]==1)])}     {len(df[df[treated]==0])}
Treated:    {len(df[(df[treated]==1) & (df[time]==0)])}     {len(df[(df[treated]==1) & (df[time]==1)])}     {len(df[df[treated]==1])}

--------------------------------------------------------
Outcome var.   | {yvar}
----------------+---------+---------+---------+---------
Before
 Control        | {mean_before_control:.3f}
 Treated        | {mean_before_treated:.3f}
 Diff (T-C)     | {diff_before:.3f}   | {se_before:.3f} | {t_before:.2f} | {p_before:.3f}

After
 Control        | {mean_after_control:.3f}
 Treated        | {mean_after_treated:.3f}
 Diff (T-C)     | {diff_after:.3f}   | {se_after:.3f} | {t_after:.2f} | {p_after:.3f}

Diff-in-Diff    | {did_coef:.3f}   | {did_se:.3f} | {did_t:.2f} | {did_p:.3f}
--------------------------------------------------------
R-square: {model.rsquared:.2f}
"""

    print(tabla)
    return model


In [12]:
res = did_diff_table(df_reg, yvar="Y", treated="treated", time="time")


DIFFERENCE-IN-DIFFERENCES ESTIMATION RESULTS
Number of observations in the DIFF-IN-DIFF: 70

            Before       After
Control:    16     24     40
Treated:    12     18     30

--------------------------------------------------------
Outcome var.   | Y
----------------+---------+---------+---------+---------
Before
 Control        | 0.358
 Treated        | 2.134
 Diff (T-C)     | 1.776   | 1.128 | 1.58 | 0.120

After
 Control        | 2.648
 Treated        | 1.904
 Diff (T-C)     | -0.744   | 0.921 | -0.81 | 0.422

Diff-in-Diff    | -2.520   | 1.456 | -1.73 | 0.088
--------------------------------------------------------
R-square: 0.08



In [13]:
import pandas as pd
import pyreadstat as st
pd.set_option('display.max_columns', None)
path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2025\2025 2T\[MT08-MT13] Microeconometría II\Clases prácticas\PS\PS3-20251111\CardKrueger1994.dta"

df, meta = st.read_dta(path)
df.head(1)

,id,t,treated,fte,bk,kfc,roys,wendys
0,1,0,1,31.0,1,0,0,0


In [14]:
meta.column_names_to_labels

{'id': 'Store ID',
 't': 'Feb. 1992 = 0; Nov. 1992 = 1',
 'treated': 'New Jersey = 1; Pennsylvania = 0',
 'fte': 'Output: Full Time Employment',
 'bk': 'Burger King == 1',
 'kfc': 'Kentuky Fried Chiken == 1',
 'roys': 'Roy Rogers == 1',
 'wendys': "Wendy's == 1"}

In [15]:
df.describe()

,id,t,treated,fte,bk,kfc,roys,wendys
count,820.000000,820.000000,820.000000,801.000000,820.000000,820.000000,820.000000,820.000000
mean,246.507317,0.500000,0.807317,17.594569,0.417073,0.195122,0.241463,0.146341
std,148.141276,0.500305,0.394647,9.022517,0.493376,0.396536,0.428232,0.353664
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,119.000000,0.000000,1.000000,11.500000,0.000000,0.000000,0.000000,0.000000
50%,237.500000,0.500000,1.000000,16.500000,0.000000,0.000000,0.000000,0.000000
75%,372.000000,1.000000,1.000000,22.000000,1.000000,0.000000,0.000000,0.000000
max,522.000000,1.000000,1.000000,80.000000,1.000000,1.000000,1.000000,1.000000
